# Recommenders 2: Pytorch and Recommenders

In this practical session, we dive a little more into [pytorch](https://pytorch.org/docs/stable/index.html) and propose to re-implement two classical matrix-factorization models with a neural network toolkit.

Also, in addition to using only rating, we propose to add text.


## WHAT IS PYTORCH?

It’s a Python-based scientific computing package targeted at two sets of audiences:

- A replacement for NumPy to use the power of GPUs
- a deep learning research platform that provides maximum flexibility and speed

### Tensors : the main unit

Tensors are similar to NumPy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

```python
#initialize an empty 5x3 matrix
x = torch.empty(5, 3)
print(x)
```

```
out[]:

tensor([[8.3665e+22, 4.5580e-41, 1.6025e-03],
        [3.0763e-41, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 3.4438e-41],
        [0.0000e+00, 4.8901e-36, 2.8026e-45],
        [6.6121e+31, 0.0000e+00, 9.1084e-44]])
        
```

### Full tutorial: 

a full pytorch tutorial can be found [here](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) do not hesitate to take a couple of minutes to skim read it. Plenty of [ressources](https://pytorch.org/resources) are available online. Also, you can have a look at the [extensive pytorch documentation](https://pytorch.org/docs/stable/index.html). 

Here, as we are defining neural networks, we mainly use the `torch.nn` module which contains most classical deep learning building blocks

### What's interesting:

Pytorch has Automatic differentiation: You only have to compute a loss function to obtain gradients automatically. How it works is detailed [here](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#pytorch-tensors-and-autograd)


# 1)  Load & Prepare Data

As usual: first thing to do is to load data: here we use an amazon review corpus

In [4]:
import gzip , json

#Here data is in json format: one "dict" per line.
def jsons2tuple(s,*keys):
    js = json.loads(s)
    return tuple([js[k] for k in keys])

#we directly read the gzip file
with gzip.open("dataset/reviews_Amazon_Instant_Video_5.json.gz","r") as f:
    data = [jsons2tuple(x,"reviewerID","asin","reviewText","overall") for x in f]

#how one exemple looks like
print("One tuple: user,item,review,rating")
print(data[:1])

train = []
test = []

#we take 80% for train and 20% for valid/test
for i,x in enumerate(data):
    if i % 8 ==0:
        test.append(x)
    else:
        train.append(x)

print(len(train))
print(len(test))

One tuple: user,item,review,rating
[('A11N155CW1UV02', 'B000H00VBQ', "I had big expectations because I love English TV, in particular Investigative and detective stuff but this guy is really boring. It didn't appeal to me at all.", 2.0)]
32485
4641


##  Prepare Data
We loaded raw data, now we prepare it:

- (1) user and items are remaped to ids from 0->len(users) /0->len(items)
- (2) reviews are tokenized using simple split

In [5]:
from collections import Counter

i_dic = {}
u_dic = {}
word_count = Counter()

prep_train = []

def text_preprocess(t):
    """
    a function to preprocess the text if needed
    takes str, returns list str
    """
    return t.split(" ")


# User and Items to key + split text + Count common words (to prune)

for uid,iid,text,rating in train:
    uk = u_dic.setdefault(uid,len(u_dic))
    ik = i_dic.setdefault(iid,len(i_dic))
    ptext = text_preprocess(text)
    word_count.update(ptext)    
    prep_train.append((uk,ik,ptext,rating))

    
# Unknown users/items are set to None    
    
prep_test = []

for uid,iid,text,rating in test:
    uk = u_dic.get(uid,None)
    ik = i_dic.get(iid,None)   
    ptext = text_preprocess(text)
    prep_test.append((uk,ik,ptext,rating))
    

# we further divide "test" in validation and test set
cutout = len(prep_test)//2
prep_val = prep_test[:cutout]
prep_test = prep_test[cutout:]
    
    

#  Pytorch Models

Now that we have loaded and prepared the data, we can define the models.


## 1) Classic SVD (with mean)

First we propose to implement a simple SVD:
### $$ \min\limits_{U,I}\sum\limits_{(u,i)} \underbrace{(r_{ui} -  (I_i^TU_u + \mu))^2}_\text{minimization} + \underbrace{\lambda(||U_u||^2+||I_u||^2 + \mu) }_\text{regularization} $$

where prediction is done in the following way:
### $$r_{ui} = \mu + U_u.I_i $$

where $\mu$ is the global mean,  $U_u$ a user embedding and $I_i$ an item embedding

### STEPS:
 To implement such model in pytorch, we need to do multiple things:
 
 - (1) model definition
 - (2) loss function
 - (3) evaluation
 - (4) training/eval loop


#### (1) Model definition

A model class typically extends `nn.Module`, the Neural network module. It is a convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.

One should define two functions: `__init__` and `forward`.

- `__init__` is used to initialize the model parameters
- `forward` is the net transformation from input to output. In fact, when doing `moduleClass(input)` you call this method.

##### (a) Initialization

Our model has different weigths:

- the user profiles (also called user embeddings) $U$
- the item profiles (also called user embeddings) $I$
- the mean bias $\mu$


##### (b) input to output operation
Technically, the prediction as defined earlier can be seen as just a dot product between two embeddings $U_u$ and $I_i$ plus the mean rating:

- `torch.sum(embed_u*embed_i,1) + self.mean` is equivalent to $r_{ui} = \mu + U_u.I_i $ 
- the `.squeeze(1)` operation is a shape operation to remove the dimension 1 (indexing starts at 0) akin to reshaping the matrix from `(batch_size,1,latent_size)` to `(batch_size,latent_size)`
- for reference, the inverse operation is `.unsqueeze()`
- we return weights to regularize them


### (TODO) Just to make sure you were following: complete the following `forward` method

In [6]:
import torch
import torch.nn as nn


# The model define as a class, inheriting from nn.Module
class ClassicMF(nn.Module):
    
    #(a) Init
    def __init__(self,nb_users,nb_items,latent_size):
        super(ClassicMF, self).__init__()
        
        #Embedding layers
        self.users = nn.Embedding(nb_users, latent_size)
        self.items = nn.Embedding(nb_items, latent_size)

        #The mean bias
        self.mean = nn.Parameter(torch.FloatTensor(1,).fill_(3))
        
        #initialize weights with very small values
        nn.init.normal_(self.users.weight,0,0.01)
        nn.init.normal_(self.items.weight,0,0.01)

    
    # (b) How we compute the prediction (from input to output)
    def forward(self, user, item): ## method called when doing ClassicMF(user,item)
        
        embed_u,embed_i = self.users(user).squeeze(1),self.items(item).squeeze(1)
        out =   ### TO COMPLETE

        return out, embed_u, embed_i, self.mean  # We return prediction + weights to regularize them
    
    


#### (2-4) full train loop

The train loop is organized around the [Dataloader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class which Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.

We just redefine a collate function

> collate_fn (callable, optional) – merges a list of samples to form a mini-batch.


**NOTE:** The dataset argument can be a list instead of a "Dataset" instance (works by duck typing)
    

##### The train loop sequence is the following:
    
[Dataset ==Dataloader==> Batch (not prepared) ==collate_fn==> Batch (prepared) ==Model.forward==> Prediction =loss_fn=> loss <-> truth 

1] PREDICT
- (a) The dataloader samples training exemples from the dataset (which is a list)
- (b) The collate_fn prepares the minibatch of training exemples
- (c) The prediction is made by feeding the minibatch in the model
- (d) The loss is computed on the prediction via a loss function

2] OPTIMIZE
- (e) Gradients are computed by automatic backard propagation
- (f) Parameters are updated using computed gradients

In [8]:
from torch.utils.data import DataLoader
import torch.nn.functional as F


# HyperParameters
n_epochs = 20
batch_size = 16
num_feat = 25
lr = 0.01
reg = 0.001


#(b) Collate function => Creates tensor batches to feed model during training
def tuple_batch(l):
    '''
    input l: list of (user,item,review, rating tuples)
    output: formatted batches (in torch tensors)

    takes n-tuples and create batch
    text -> seq word #id
    '''
    users, items, _,ratings = zip(*l) #we ignore review text
    users_t = torch.LongTensor(users)
    items_t = torch.LongTensor(items)
    ratings_t = torch.FloatTensor(ratings)
    
    return users_t, items_t, ratings_t
    


#(d) Loss function => Combines MSE and L2
def loss_func(pred,ratings_t,reg,*params):
    '''
    mse loss combined with l2 regularization.
    params assumed 2-dimension
    '''
    mse = F.mse_loss(pred,ratings_t,size_average=False)
    l2 = 0
    for p in params:
        l2 += torch.mean(p.norm(2,-1))
        
    return (mse/pred.size(0)) + reg*l2 , mse
    
#
# Training script starts here
#    


model = ClassicMF(len(u_dic),len(i_dic),num_feat)

# (a) dataloader will sample data from datasets using collate_fn tuple_batch
dataloader_train = DataLoader(prep_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=tuple_batch)
dataloader_val = DataLoader(prep_val, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=tuple_batch)
dataloader_test = DataLoader(prep_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=tuple_batch)

optimizer = torch.optim.SGD(model.parameters(),lr=lr)

# Train loop
for e in range(n_epochs):
    mean_loss = [0,0,0] #train/val/test

    ## Training loss (the one we train with)
    
    for users_t,items_t,ratings_t in dataloader_train:
        model.train() # set the model on train mode
        model.zero_grad() # reset gradients
        
        #(c) predictions are made by the model
        pred,*params = model(users_t,items_t)
        
        #(d) loss computed on predictions, we added regularization
        loss,mse_loss = loss_func(pred,ratings_t,reg,*params)
        
        loss.backward() #(e) backpropagating to get gradients
        
        mean_loss[0] += mse_loss
        optimizer.step() #(f) updating parameters
    
    ## Validation loss (no training)

    for users_t,items_t,ratings_t in dataloader_val:
        model.eval() # Inference mode
        pred,*params = model(users_t,items_t)
        _,mse_loss = loss_func(pred,ratings_t,reg,*params)
    
        mean_loss[1] += mse_loss    
        
    ## Test loss (no training)
        
    for users_t,items_t,ratings_t in dataloader_test:
        model.eval()
        pred,*params = model(users_t,items_t)
        _,mse_loss = loss_func(pred,ratings_t,reg,*params)
    
        mean_loss[2] += mse_loss    

    print("-"*25)
    print("epoch",e, "mse (train/val/test)", round((mean_loss[0]/len(prep_train)).item(),3),"/",  round((mean_loss[1]/len(prep_val)).item(),3),"/",  round((mean_loss[2]/len(prep_test)).item(),3))
    
    



-------------------------
epoch 0 mse (train/val/test) 1.271 / 1.101 / 1.398
-------------------------
epoch 1 mse (train/val/test) 1.252 / 1.109 / 1.394
-------------------------
epoch 2 mse (train/val/test) 1.252 / 1.092 / 1.407
-------------------------
epoch 3 mse (train/val/test) 1.252 / 1.094 / 1.404
-------------------------
epoch 4 mse (train/val/test) 1.252 / 1.101 / 1.399
-------------------------
epoch 5 mse (train/val/test) 1.252 / 1.094 / 1.405
-------------------------
epoch 6 mse (train/val/test) 1.252 / 1.097 / 1.401
-------------------------
epoch 7 mse (train/val/test) 1.252 / 1.099 / 1.4
-------------------------
epoch 8 mse (train/val/test) 1.252 / 1.096 / 1.402
-------------------------
epoch 9 mse (train/val/test) 1.252 / 1.091 / 1.409


KeyboardInterrupt: 

## (Your turn) Koren 2009 model:

Here, this model simply adds a bias for each user and for each item

### $$ \min\limits_{U,I}\sum\limits_{(u,i)} \underbrace{(r_{ui} -  (I_i^TU_u + \mu+ \mu_i+\mu_u))^2}_\text{minimization} + \underbrace{\lambda(||U_u||^2+||I_u||^2 + \mu  + \mu+ \mu_i+\mu_u) }_\text{regularization} $$


### $$r_{ui} = \mu + \mu_i + \mu_u + U_u.I_i $$

### TODO:

- (a) complete the model initialization
- (b) complete the forward method

In [15]:
class KorenMF(nn.Module):

    def __init__(self,nb_users,nb_items,latent_size):
        super(KorenMF, self).__init__()
        
        self.users = ##
        self.items = ###
        self.umean = ###
        self.imean = ###
        self.gmean =  ###

        nn.init.normal_(self.users.weight,0,0.01)
        nn.init.normal_(self.items.weight,0,0.01)
        nn.init.normal_(self.umean.weight,0.5,1)
        nn.init.normal_(self.imean.weight,0.5,1)
        
        
    def forward(self, user,item):
        embed_u,embed_i = self.users(user).squeeze(1) , self.items(item).squeeze(1)
        umean, imean = self.umean(user).squeeze(-1) , self.imean(item).squeeze(-1)
        
        out = ##############

        return out , embed_u, embed_i, umean , imean , self.gmean

### (TODO) Here, train loop stays the same, you only have to change the model

In [16]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

n_epochs = 50
batch_size = 16
num_feat = 25
lr = 0.01
reg = 0.001



def tuple_batch(l):
    '''
    input l: list of (user,item,review, rating tuples)
    output: formatted batches (in torch tensors)

    takes n-tuples and create batch
    text -> seq word #id
    '''
    users, items, _ ,ratings = zip(*l) # we ignore reviews for now
    users_t = torch.LongTensor(users)
    items_t = torch.LongTensor(items)
    ratings_t = torch.FloatTensor(ratings)
    
    return users_t,items_t,ratings_t


def loss_func(pred,ratings_t,reg,*params):
    '''
    mse loss combined with l2 regularization.
    params assumed 2-dimension
    '''
    mse = F.mse_loss(pred,ratings_t,size_average=False)
    l2 = 0
    for p in params:
        l2 += torch.mean(p.norm(2,-1))
        
    return (mse/pred.size(0)) + reg*l2 , mse
    

model =  ## TO COMPLETE


dataloader_train = DataLoader(prep_train, batch_size=batch_size, shuffle=True, num_workers=3, collate_fn=tuple_batch)
dataloader_val = DataLoader(prep_val, batch_size=batch_size, shuffle=True, num_workers=3, collate_fn=tuple_batch)
dataloader_test = DataLoader(prep_test, batch_size=batch_size, shuffle=False, num_workers=3, collate_fn=tuple_batch)

optimizer = torch.optim.SGD(model.parameters(),lr=lr)


for e in range(n_epochs):
    mean_loss = [0,0,0] #train/val/test

    for users_t,items_t,ratings_t in dataloader_train:
        model.train()
        model.zero_grad()
        pred,*params = model(users_t,items_t)

        loss,mse_loss = loss_func(pred,ratings_t,reg,*params)
        loss.backward()
        
        mean_loss[0] += mse_loss
        optimizer.step()
    
    

    for users_t,items_t,ratings_t in dataloader_val:
        model.eval()
        pred,*params = model(users_t,items_t)
        _,mse_loss = loss_func(pred,ratings_t,reg,*params)
    
        mean_loss[1] += mse_loss    
        
    for users_t,items_t,ratings_t in dataloader_test:
        model.eval()
        pred,*params = model(users_t,items_t)
        _,mse_loss = loss_func(pred,ratings_t,reg,*params)
    
        mean_loss[2] += mse_loss    

    print("-"*25)
    print("epoch",e, "mse (train/val/test)", round((mean_loss[0]/len(prep_train)).item(),3),"/",  round((mean_loss[1]/len(prep_val)).item(),3),"/",  round((mean_loss[2]/len(prep_test)).item(),3))
    
    

-------------------------
epoch 0 mse (train/val/test) 3.028 / 2.947 / 2.978
-------------------------
epoch 1 mse (train/val/test) 2.837 / 2.883 / 2.78
-------------------------
epoch 2 mse (train/val/test) 2.702 / 2.751 / 2.665
-------------------------
epoch 3 mse (train/val/test) 2.596 / 2.684 / 2.561
-------------------------
epoch 4 mse (train/val/test) 2.51 / 2.606 / 2.496
-------------------------
epoch 5 mse (train/val/test) 2.435 / 2.545 / 2.433
-------------------------
epoch 6 mse (train/val/test) 2.37 / 2.486 / 2.386
-------------------------
epoch 7 mse (train/val/test) 2.31 / 2.442 / 2.334
-------------------------
epoch 8 mse (train/val/test) 2.256 / 2.413 / 2.296
-------------------------
epoch 9 mse (train/val/test) 2.206 / 2.345 / 2.258
-------------------------
epoch 10 mse (train/val/test) 2.159 / 2.3 / 2.226
-------------------------
epoch 11 mse (train/val/test) 2.116 / 2.263 / 2.192
-------------------------
epoch 12 mse (train/val/test) 2.075 / 2.223 / 2.166
--

Process Process-366:
Process Process-364:
  File "/home/charles/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
Traceback (most recent call last):
Process Process-365:
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/process.py", line 93

KeyboardInterrupt: 

  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
  File "/home/charles/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/charles/anaconda3/lib/p